In [1]:
!pip install mediapipe
!pip install opencv-python


DEPRECATION: Loading egg at c:\users\franf\appdata\local\programs\python\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


DEPRECATION: Loading egg at c:\users\franf\appdata\local\programs\python\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [9]:
import cv2
import mediapipe as mp
import json

# Inicializar BlazePose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Cargar la imagen
image = cv2.imread('../yolo/videoframe_0.jpg')
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Realizar la detección de pose
results = pose.process(image_rgb)

blazepose_results = []

# Dibujar las anotaciones de la pose en la imagen
if results.pose_landmarks:
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    #guardar en annotations
    keypoints = []
    for landmark in results.pose_landmarks.landmark:
        keypoints.extend([landmark.x * image.shape[1], landmark.y * image.shape[0], landmark.visibility])
    
    ann = {
        "image_id": "videoframe",
        "category_id": 1,  # Persona
        "keypoints": keypoints,
        "score": 1.0  # Puedes ajustar el score según sea necesario
    }
    blazepose_results.append(ann)

#guardar la img
cv2.imwrite('testBlazePose.jpg', image)
# Mostrar la imagen con las poses detectadas
cv2.imshow('BlazePose Result', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Liberar recursos
pose.close()

# Guardar las anotaciones en un archivo JSON
with open('blazepose_results.json', 'w') as f:
    json.dump(blazepose_results, f)


c:\Users\franf\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [11]:
#usando el dataset de wholeBody
import os

# Inicializar BlazePose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Directorio de imágenes y archivo de anotaciones
image_dir = '../../val2017/val2017'  # Ajusta según la estructura de tu directorio
#annotation_file = '../../coco_wholebody_val_v1.0.json'
annotation_file = '../../person_keypoints_val2017.json'
blazepose_results = []

# Cargar el dataset COCO
with open(annotation_file, 'r') as f:
    coco_data = json.load(f)

# Procesar las imágenes
for img_info in coco_data['images']:
    image_path = os.path.join(image_dir, img_info['file_name'])
    image = cv2.imread(image_path)
    if image is None:
        continue

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        keypoints = []
        for landmark in results.pose_landmarks.landmark:
            keypoints.extend([landmark.x * image.shape[1], landmark.y * image.shape[0], landmark.visibility])
        
        ann = {
            "image_id": img_info['id'],
            "category_id": 1,  # Persona
            "keypoints": keypoints,
            "score": 1.0  # Puedes ajustar el score según sea necesario
        }
        blazepose_results.append(ann)

pose.close()

# Guardar las anotaciones en un archivo JSON
with open('blazepose_results-wholeBody.json', 'w') as f:
    json.dump(blazepose_results, f)

#guardar una img para ver como la analiza blazepose
cv2.imwrite('testBlazePoseWholeBody.jpg', image) #seria la ultima imagen procesada del dataset


c:\Users\franf\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


True

In [16]:
import cv2
import mediapipe as mp
import json
import os
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

# Inicializar BlazePose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Directorio de imágenes y archivo de anotaciones
image_dir = '../../val2017/val2017'  # Ajusta según la estructura de tu directorio
annotation_file = '../yolo/datasets/coco-pose/annotations/person_keypoints_val2017.json'
blazepose_results = []

# Cargar el dataset COCO
with open(annotation_file, 'r') as f:
    coco_data = json.load(f)
coco = COCO(annotation_file)

# Función para convertir 33 keypoints de BlazePose a 17 keypoints de COCO
def convert_blazepose_to_coco(landmarks, image_shape):
    keypoints = [None] * 17
    keypoints[0] = landmarks[0]  # Nose
    keypoints[1] = landmarks[1]  # Left eye
    keypoints[2] = landmarks[4]  # Right eye
    keypoints[3] = landmarks[7]  # Left ear
    keypoints[4] = landmarks[8]  # Right ear
    keypoints[5] = landmarks[11]  # Left shoulder
    keypoints[6] = landmarks[12]  # Right shoulder
    keypoints[7] = landmarks[13]  # Left elbow
    keypoints[8] = landmarks[14]  # Right elbow
    keypoints[9] = landmarks[15]  # Left wrist
    keypoints[10] = landmarks[16]  # Right wrist
    keypoints[11] = landmarks[23]  # Left hip
    keypoints[12] = landmarks[24]  # Right hip
    keypoints[13] = landmarks[25]  # Left knee
    keypoints[14] = landmarks[26]  # Right knee
    keypoints[15] = landmarks[27]  # Left ankle
    keypoints[16] = landmarks[28]  # Right ankle
    coco_keypoints = []
    for point in keypoints:
        if point is not None:
            coco_keypoints.extend([point.x * image_shape[1], point.y * image_shape[0], point.visibility])
        else:
            coco_keypoints.extend([0, 0, 0])  # Fill with zeros if no point detected
    return coco_keypoints

# Procesar las imágenes
for img_info in coco_data['images']:
    image_path = os.path.join(image_dir, img_info['file_name'])
    image = cv2.imread(image_path)
    if image is None:
        continue

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        # Dibujar las anotaciones de la pose en la imagen
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        keypoints = convert_blazepose_to_coco(results.pose_landmarks.landmark, image.shape)
        
        ann = {
            "image_id": img_info['id'],
            "category_id": 1,  # Persona
            "keypoints": keypoints,
            "score": 1.0  # Puedes ajustar el score según sea necesario
        }
        blazepose_results.append(ann)

# Guardar las anotaciones en un archivo JSON
with open('blazepose_results-wholeBody.json', 'w') as f:
    json.dump(blazepose_results, f)

# Guardar y mostrar una imagen procesada
if 'image' in locals():
    # Guardar la última imagen procesada con las anotaciones
    cv2.imwrite('testBlazePoseWholeBody.jpg', image)
    # Mostrar la imagen con las poses detectadas
    cv2.imshow('BlazePose WholeBody Result', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Liberar recursos
pose.close()

# Evaluar las métricas de rendimiento usando pycocotools
coco_dt = coco.loadRes('blazepose_results-wholeBody.json')
coco_eval = COCOeval(coco, coco_dt, 'keypoints')
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()

# Imprimir métricas relevantes
print("mAP: ", coco_eval.stats[0])
print("AP@0.50: ", coco_eval.stats[1])
print("AP@0.75: ", coco_eval.stats[2])
print("AP (M): ", coco_eval.stats[3])
print("AP (L): ", coco_eval.stats[4])
print("AR: ", coco_eval.stats[8])

print("Anotaciones guardadas en: blazepose_results-wholeBody.json")
if 'image' in locals():
    print("Imagen procesada guardada en: testBlazePoseWholeBody.jpg")


loading annotations into memory...
Done (t=0.66s)
creating index...
index created!


c:\Users\franf\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Loading and preparing results...
DONE (t=0.12s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *keypoints*
DONE (t=0.86s).
Accumulating evaluation results...
DONE (t=0.06s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets= 20 ] = 0.088
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets= 20 ] = 0.164
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets= 20 ] = 0.087
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets= 20 ] = 0.029
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets= 20 ] = 0.170
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 20 ] = 0.132
 Average Recall     (AR) @[ IoU=0.50      | area=   all | maxDets= 20 ] = 0.201
 Average Recall     (AR) @[ IoU=0.75      | area=   all | maxDets= 20 ] = 0.140
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets= 20 ] = 0.039
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDe